In [1]:
path = 'patches500'

In [2]:
import os
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
import pandas as pd

In [3]:
def list_flatten(lis):
    flatList = []
    for element in lis:
        if type(element) is list:
            for item in element:
                flatList.append(item)
        else:
            flatList.append(element)
    return flatList

In [ ]:
files = os.listdir(path)
df = pd.DataFrame({'files': files})
df['patient'] = df['files'].apply(lambda x: x.split('_')[0])
df['files'] = df['files'].apply(lambda x: os.path.join(path,x))
df = df.groupby('patient')['files'].apply(list).reset_index(name="files")
df['label'] = df['files'].apply(lambda x: int(x[0].split('_')[4]))
df = df[df.label !=6]
df = df[df.label !=7]

df = df.reset_index(drop=True)

data_counts = {}
data_list = {}
max_length = 0
train_patients, valid_patients = [], []

for sub_df in df.groupby('label'):
    train_split, valid_split = train_test_split(sub_df[1], test_size=0.2, random_state=42)

    train_list = list_flatten(train_split['files'].tolist())
    valid_list = list_flatten(valid_split['files'].tolist())

    train_patients+= train_list
    valid_patients+= valid_list
    
    data_counts[sub_df[0]] = len(train_list)
    data_list[sub_df[0]] = train_list
    if len(train_list)>max_length:
        max_length = len(train_list)

# balance data
# for i,j in data_counts.items():
#     m = round(max_length/j)
#     train_patients+= data_list[i]*m


In [5]:
train_patients = pd.DataFrame(train_patients, columns=['path'])
valid_patients = pd.DataFrame(valid_patients, columns=['path'])

train_patients['patches'] = train_patients['path'].apply(lambda x: '_'.join(x.split('/')[1].split('_')[:-3]))
valid_patients['patches'] = valid_patients['path'].apply(lambda x: '_'.join(x.split('/')[1].split('_')[:-3]))

train_patients = train_patients.sort_values('path').reset_index(drop=True)
valid_patients = valid_patients.sort_values('path').reset_index(drop=True)

train_df = train_patients['path'].groupby(train_patients.patches).apply(list).reset_index()
valid_df = valid_patients['path'].groupby(valid_patients.patches).apply(list).reset_index()

train_df['count_patches'] = train_df['path'].apply(lambda x: len(x))
valid_df['count_patches'] = valid_df['path'].apply(lambda x: len(x))

In [6]:
train_df.to_json('train_df.json')
valid_df.to_json('valid_df.json')